# DATE COLUMN DETECTION - GANESH RAM GURURAJAN

There are two functions :
1. **getDateColumns()**
2. **featureDateDifferences()**


1. **getDateColumns() - Explanation**

This method finds columns containing 'DATE' values based on calculating the possibilities. Firstly, the column name is checked, if it contains the string 'date' in it, the possibility is more. But, sometimes we get features like 'price as of date', in which case, it might be a numeric column.

Secondly, all non_numerical columns are checked to have two **'/ '** or two **'white spaces'** or two **' - '** or two **' . '** . Also, we check if there's any name of month present in it, like **' jan ' , ' dec '** and so on. As these criteria get satisfied, the possbility is increased by a small fraction , else, possibility of that non-numerical column is reduced. At the end of iteration, we definitely have a few columns who possibility is more than 1, and all others below 1.0. These are the **DATE_COLUMNS**.

2. **featureDateDifferences() - Explanation**

This method adds new features which are the difference between the dates, while taking two at a time. 

Now, we have a list of Date Columns from the previous method. We need Difference Columns. We need to know how many differences we can make. For instance, if we have 2 date columns, we can feature 1 difference column, and if we have 6 date columns, we can feature 15 difference columns.

Example: If Date Columns = [D1,D2,D3], then difference columns = [D1-D2,D1-D3,D2-D3]. Now, for every column in difference columns, get the difference from the required couple of Date Columns.

Till now, only columns were validated, now every entry from the Date Columns will be validated, using **REGULAR EXPRESSIONS**. Finally the date is returned by calculating date(column1) - date(column2) using **DATE Module**  for each New Difference Feature.

***Important***: The **REGEX** itself gives us a None, if the DATA isn't a date type, but it doesn't check for something like 29/feb/2019 where 2019 is not a leap year. Otherwise, all kinds of invalidity is checked efficiently.

***Note***: Similar to **NaN**, **NaT** is also sensed by **df.isnull()** method - Checked. So, dropping missing values is easier

### Initial Steps - Problem 1
### Package Import, Creation of dummy datasets, shuffling



In [1]:
# Packages Import
import pandas as pd
import numpy as np
from datetime import date
import random
import re

In [2]:
# Creating Date Column 1
dateData1 = ['01/01/2020','01.Feb.2020','01 mar 2015','2016 Jun 15','2016.06.15',np.nan,'1975/dec/02','1975 12 02','abc','1950.02.02']

In [3]:
# Creating Dummy Column 1
dummyData1 = [np.random.rand() for _ in range(len(dateData1))]

In [4]:
# Creating Dummy Column 2
dummyData2 = ['abc','gbvc','7yg','np.nan','someRandomText','hgufg','3y5g','bh4','4fa','/759']

In [5]:
# Creating Date Column 2
dateData2 = dateData1[:]
random.shuffle(dateData2)

In [6]:
x = dateData1[:]
y = dateData1[:]
random.shuffle(x)
random.shuffle(y)

In [7]:
# DataFrame from all above lists
df = pd.DataFrame(list(zip(dateData1,dateData2,dummyData1,dummyData2,x,y)), 
               columns =['Date1', 'date2','Price as of date','dummy2','X','Y'])

### My Dummy DataFrame that contains different columns and different date formats, along with invalid data including np.nan - Problem 1

In [8]:
# The DataFrame with which I'd like to work
df

,Date1,date2,Price as of date,dummy2,X,Y
0,01/01/2020,01 mar 2015,0.498692,abc,2016 Jun 15,01 mar 2015
1,01.Feb.2020,1975 12 02,0.703615,gbvc,01.Feb.2020,abc
2,01 mar 2015,2016.06.15,0.864557,7yg,2016.06.15,01.Feb.2020
3,2016 Jun 15,2016 Jun 15,0.193985,np.nan,1975 12 02,2016 Jun 15
4,2016.06.15,1950.02.02,0.878020,someRandomText,01/01/2020,01/01/2020
5,NaN,abc,0.381382,hgufg,1950.02.02,1975 12 02
6,1975/dec/02,NaN,0.792935,3y5g,01 mar 2015,2016.06.15
7,1975 12 02,01/01/2020,0.333762,bh4,NaN,1975/dec/02
8,abc,01.Feb.2020,0.865258,4fa,1975/dec/02,1950.02.02
9,1950.02.02,1975/dec/02,0.360818,/759,abc,NaN


### **The Functions** - Problem 1


In [9]:
######################## ----------------- FIRST FUNCTION --------------------- ######################

# Global List of all months
months = ['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']


def getDateColumns(df,withPossibilies=0):

  '''
  This method Identifies all columns with 'DATE' data by maximizing out the possibilities
  '''

  # First get all non-numerical Columns
  numerical_cols = df._get_numeric_data().columns
  non_numeric_cols = list(set(df.columns) - set(numerical_cols))


  # This dictionary stores possibilities of a column containing 'DATES' based on the name of the column
  Possibility = {}
  for column in non_numeric_cols:
    if 'date' in column.lower():                  
      Possibility[column] = 1
    else:
      Possibility[column] = 0

    for entry in df[column]:                                                    # ITERATE THROUGH EVERY ENTRY AND TRY SPLITTING THE VALUE AND INCREMENT/DECREMENT POSSIBILITY 
      try:                                                                      # USING EXCEPTION HANDLING

        if len(entry.split('/')) == 3 or len(entry.split('-')) == 3 or len(entry.split(' ')) == 3 or len(entry.split('.')) == 3:
          Possibility[column] += 0.357

          for month in months:
            if month in entry.lower():
              Possibility[column] += 0.357            
     
      except (ValueError,AttributeError) as e:
        Possibility[column] -= 0.357        

  # This contains the final DATE Columns
  DATE_COLUMNS = []
  for key,value in Possibility.items():             
    if value > 1:                                                               # IF THE POSSIBILITY OF THE COLUMN IN GREATER THAN 1, THEN IT IS DEFINITELY A 'DATE COLUMN'
      DATE_COLUMNS.append(key)    

  if not withPossibilies:
    return DATE_COLUMNS
  else:
    return DATE_COLUMNS,Possibility

In [10]:
######################## ----------------- SECOND FUNCTION -------------------- ########################

def featureDateDifferences(df,DATE_COLUMNS):

  '''
  Creates new features containing difference between date columns while taking two columns at a time. If no columns are created, it means there's one or less number of date columns
  '''

  number_of_diff_columns = 0

  if len(DATE_COLUMNS) > 1:                                                     # At least two columns are required for making a difference
    for i in range(1,len(DATE_COLUMNS)):
      number_of_diff_columns += i                                               # If number of date columns is 6, we can make 15 difference columns taking 2 date columns at a time
                            
  featureDictionary = {}

  # Name New Features
  try:
    for i in range(len(DATE_COLUMNS)):
      for j in range(i+1,len(DATE_COLUMNS)):
        newFeature = DATE_COLUMNS[i] + '_minus_' + DATE_COLUMNS[j]              #Name New difference columns based on names of exiting date columns
        featureDictionary[newFeature] = [DATE_COLUMNS[i],DATE_COLUMNS[j]]
  except IndexError:
    pass



  ######### -------------- REGULAR EXPRESSIONS------------ ############



  # REGULAR EXPRESSIONS
  def returnFormat(string,delimiter):

    '''
    Checks if there's any match for date format
    '''
    string = string.lower()

    # REGULAR EXPRESSIONS
    dmy_num = re.search("^([1-9]|0[1-9]|1[0-9]|2[0-9]|3[0-1])(\.|-|/| )([1-9]|0[1-9]|1[0-2])(\.|-|/| )([0-9][0-9]|19[0-9][0-9]|20[0-9][0-9])$",string)
    ymd_num = re.search("^([0-9][0-9]|19[0-9][0-9]|20[0-9][0-9])(\.|-|/| )([1-9]|0[1-9]|1[0-2])(\.|-|/| )([1-9]|0[1-9]|1[0-9]|2[0-9]|3[0-1])$",string)
    
    dmy_mon = re.search("^([1-9]|0[1-9]|1[0-9]|2[0-9]|3[0-1])(\.|-|/| )(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)(\.|-|/| )([0-9][0-9]|19[0-9][0-9]|20[0-9][0-9])$",string)
    ymd_mon = re.search("^([0-9][0-9]|19[0-9][0-9]|20[0-9][0-9])(\.|-|/| )(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)(\.|-|/| )([1-9]|0[1-9]|1[0-9]|2[0-9]|3[0-1])$",string)

    if ymd_num:
      return string.split(delimiter)
    elif dmy_num:
      string = string.split(delimiter)
      string.reverse()
      return string
    elif ymd_mon:
      string = string.split(delimiter)
      string[1] = months.index(string[1]) + 1
      return string
    elif dmy_mon:
      string = string.split(delimiter)
      string[1] = months.index(string[1]) + 1
      string.reverse()
      return string
    else:
      return None                                                               # If no regex matches, it's an invalid input in a date column, hence returns 'None'
      


########### Apply function to new difference features by taking two date columns at a time ############


  # Iterate Through all DATE COLUMNS
  def iterateThrough(cols):

    '''
    Iterate through DATE_COLUMNS for every new difference column
    '''

    f_date = str(cols[0])
    l_date = str(cols[1])

    # Select the delimiter and pass it along with the string to enable .split() method
    if f_date.count('/') == 2:
      f_date = returnFormat(f_date,'/')                                         # The delimiter in the date can be a '/'
    elif f_date.count('-') == 2:
      f_date = returnFormat(f_date,'-')                                         # or a '-'
    elif f_date.count(' ') == 2:
      f_date = returnFormat(f_date,' ')                                         # or a space
    elif f_date.count('.') == 2:
      f_date = returnFormat(f_date,'.')                                         # even a '.' as found in various date formats
    else:
      f_date = None                                                             # If nothing is found, this becomes an invalid entry

    if l_date.count('/') == 2:
      l_date = returnFormat(l_date,'/')
    elif l_date.count('-') == 2:
      l_date = returnFormat(l_date,'-')
    elif l_date.count(' ') == 2:
      l_date = returnFormat(l_date,' ')
    elif l_date.count('.') == 2:
      l_date = returnFormat(l_date,'.')
    else:
      l_date = None


    if (f_date == None or l_date == None):
      return np.nan
    else:
      first = date(int(f_date[0]),int(f_date[1]),int(f_date[2]))
      last = date(int(l_date[0]),int(l_date[1]),int(l_date[2]))
      return first-last


  #APPLY FUNCTION FOR EVERY NEW FEATURE FROM EXISTING COUPLES OF DATE COLUMNS FROM DICTIONARY
  for new_column,existing_columns in featureDictionary.items():
    df[new_column] = df[existing_columns].apply(iterateThrough,axis=1)
  

  # RETURN THE NEW DATAFRAME with NEW_FEATURES containing DATE DIFFERENCES
  return df

### OUTPUT - Problem 1

In [11]:
# This is the result
# CONTAINS ORIGINAL DATA ALONG WITH NEW FEATURES CONTAINING THE WORD ** MINUS ** IN IT

# ***********IMPORTANT***********
# There is NaT in the output because there is invalid entry in date columns

date_cols,poss = getDateColumns(df,1)                                           # Second Parameter to get possibilies from the function, can be 0, if we don't need
print('The Date Columns are {}'.format(date_cols))
print('The possibilities Calculated are {}'.format(poss))

df = featureDateDifferences(df,date_cols)                                       # First parameter is DataFrame itself, and second DATE_COLUMNS
                                                                                # As expected the 'dummy2' column has zero possibility of being a date column, as shown below
                                                                                # Whereas all others have a positive possiblity

The Date Columns are ['date2', 'Date1', 'Y', 'X']
The possibilities Calculated are {'date2': 4.927000000000001, 'dummy2': 0, 'Date1': 4.927000000000001, 'Y': 3.9270000000000005, 'X': 3.9270000000000005}


In [12]:
# Output DataFrame
df

,Date1,date2,Price as of date,dummy2,X,Y,date2_minus_Date1,date2_minus_Y,date2_minus_X,Date1_minus_Y,Date1_minus_X,Y_minus_X
0,01/01/2020,01 mar 2015,0.498692,abc,2016 Jun 15,01 mar 2015,-1767 days,0 days,-472 days,1767 days,1295 days,-472 days
1,01.Feb.2020,1975 12 02,0.703615,gbvc,01.Feb.2020,abc,-16132 days,NaT,-16132 days,NaT,0 days,NaT
2,01 mar 2015,2016.06.15,0.864557,7yg,2016.06.15,01.Feb.2020,472 days,-1326 days,0 days,-1798 days,-472 days,1326 days
3,2016 Jun 15,2016 Jun 15,0.193985,np.nan,1975 12 02,2016 Jun 15,0 days,0 days,14806 days,0 days,14806 days,14806 days
4,2016.06.15,1950.02.02,0.878020,someRandomText,01/01/2020,01/01/2020,-24240 days,-25535 days,-25535 days,-1295 days,-1295 days,0 days
5,NaN,abc,0.381382,hgufg,1950.02.02,1975 12 02,NaT,NaT,NaT,NaT,NaT,9434 days
6,1975/dec/02,NaN,0.792935,3y5g,01 mar 2015,2016.06.15,NaT,NaT,NaT,-14806 days,-14334 days,472 days
7,1975 12 02,01/01/2020,0.333762,bh4,NaN,1975/dec/02,16101 days,16101 days,NaT,0 days,NaT,NaT
8,abc,01.Feb.2020,0.865258,4fa,1975/dec/02,1950.02.02,NaT,25566 days,16132 days,NaT,NaT,-9434 days
9,1950.02.02,1975/dec/02,0.360818,/759,abc,NaN,9434 days,NaT,NaT,NaT,NaT,NaT
